### Reported deaths map

Import the needed modules and paths

In [2]:
%run ../code/conf_files.ipynb

Read the [geojson dataset](../Data/datasets.ipynb#eng) (with the geometry) and the [preL_dt](..Data/datasets.ipynb#preL_dt) (with the information about reported deaths that I want to map)

In [3]:
england = open("../Data/NHS_England_Regions_(April_2020)_Boundaries_EN_BFC.geojson", "r")
england = england.read()
england = json.loads(england)
preL_dt = pd.read_csv("%s/26-4-2020_yyAIR_COVID_PRE_LD_dt.csv" %path)

There must be a common id to preL_dt and england. I want to use the property 'objectid' for this purpose, as it is unique for each region. 
To create a match between england and preL_dt I create a third element in each feature with the id and then a new column in preL_dt with the correponding id for each region. 


preL_dt.Region and the property 'nhser20nm' in each feature both store the name of the region. I need them to have the the region name written in the same way to be able to compare them.

In [4]:
preL_dt.loc[preL_dt['Region'] =='East Of England','Region'] = 'East of England'
preL_dt.loc[preL_dt['Region'] =='North East And Yorkshire','Region'] = 'North East and Yorkshire'

In [5]:
area={}
for feature in england['features']:
    feature['id']=feature['properties']['objectid']
    area[feature['properties']['nhser20nm']]=feature['id']
preL_dt['id']=preL_dt['Region'].apply(lambda x: area[x])

Now I have all the objects I need to draw the map.

In [6]:
fig_deaths = px.choropleth_mapbox(preL_dt, 
                           locations="id", 
                           featureidkey='properties.objectid', 
                           geojson=england, color='Deaths', 
                           hover_name='Region', 
                           mapbox_style='white-bg', 
                           zoom=4.5, 
                           center = {"lat": 53, "lon": 0},
                          color_continuous_scale = 'blues',
                          title = 'COVID-19 (reported deaths)')
#fig_deaths.show()
plotly.offline.plot(fig_deaths, filename='../data_out/map_deaths.html')

'../data_out/map_deaths.html'